In [8]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langgraph langgraph-prebuilt langgraph_sdk langgraph-checkpoint-sqlite langsmith langchain-community tavily-python wikipedia

In [9]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
import operator
from typing import List, Annotated, TypedDict
from langchain_core.messages import AnyMessage, HumanMessage
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages

# 1. Defining The State

class TravelAgentState(TypedDict):
    
    
    messages: Annotated[List[AnyMessage], add_messages] 
    
    
    destination: str
    budget: str
    travel_dates: str
    
    flight_options: Annotated[List[dict], operator.add]
    train_options: Annotated[List[dict], operator.add]
    bus_options: Annotated[List[dict], operator.add]
    accommodation_options: Annotated[List[dict], operator.add]


In [11]:
# 2. Creating a Test Node

def print_node(state: TravelAgentState):
    last_message = state['messages'][-1].content
    print(f"Last message received: {last_message}")
    return

In [12]:
# 3. Setting Up Memory
import sqlite3
conn = sqlite3.connect(":memory:", check_same_thread = False)
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver(conn)

In [13]:
# 4. Building The Graph
builder = StateGraph(TravelAgentState)

builder.add_node("print", print_node)
builder.add_edge(START, "print")
builder.add_edge("print", END)

app = builder.compile(checkpointer=memory)


In [14]:
# 5. Testing The Graph

config = {"configurable": {"thread_id": "user-2"}}

first_input = {"messages": [HumanMessage(content="Hi! I'm planning a trip.")]}
response = app.invoke(first_input, config)

second_input = {"messages": [HumanMessage(content="Can you remember my last message?")]}
response = app.invoke(second_input, config)


final_state = app.get_state(config)

print(final_state.values['messages'])

Last message received: Hi! I'm planning a trip.
Last message received: Can you remember my last message?
[HumanMessage(content="Hi! I'm planning a trip.", additional_kwargs={}, response_metadata={}, id='10fd51ab-1994-4c83-9e7d-6ede4a7494e9'), HumanMessage(content='Can you remember my last message?', additional_kwargs={}, response_metadata={}, id='11dbd914-babe-40df-9ee0-32ee6bbf1ce0')]
